In [84]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
from functools import partial

In [85]:
from geopy.geocoders import ArcGIS
from geopy.extra.rate_limiter import RateLimiter

In [86]:
NFL = pd.read_csv('NFL-player-bio.csv')
NFL['Full Address'] = NFL['Place_Birth'] + ', ' + NFL['State']
NFL['College'] = NFL['College'] + " University" 
NFL.head(10)

,Name,Date_Birth,Place_Birth,State,College,Experience,HS,HOF,Full Address
0,Gene Ronzani,28/3/1909,Iron Mountain,MI,Marquette University,8 Seasons,NaN,NaN,"Iron Mountain, MI"
1,Keith Molesworth,20/10/1905,Washington,IA,Monmouth University,7 Seasons,NaN,NaN,"Washington, IA"
2,Carl Brumbaugh,22/9/1906,West Milton,OH,Florida University,8 Seasons,NaN,NaN,"West Milton, OH"
3,Ken Strong,21/3/1906,West Haven,CT,N.Y.U. University,12 Seasons,NaN,1967.0,"West Haven, CT"
4,George Corbett,14/6/1908,Dix,IL,Millikin University,7 Seasons,NaN,NaN,"Dix, IL"
5,Bronko Nagurski,3/11/1908,Rainy River,ON,Minnesota University,9 Seasons,NaN,1963.0,"Rainy River, ON"
6,Solly Sherman,25/9/1917,Chicago,IL,Chicago University,2 Seasons,NaN,NaN,"Chicago, IL"
7,Bob Snyder,6/2/1913,Fremont,OH,Ohio U. University,6 Seasons,NaN,NaN,"Fremont, OH"
8,Frank Filchock,8/10/1916,Crucible,PA,Indiana University,8 Seasons,NaN,NaN,"Crucible, PA"
9,Hank Soar,17/8/1914,Alton,RI,Providence University,9 Seasons,NaN,NaN,"Alton, RI"


In [87]:
geolocator = ArcGIS(user_agent="NFL-GPS")

geocode = RateLimiter(geolocator.geocode, min_delay_seconds= 1)

In [88]:
# GPS = pd.DataFrame(columns= ['Latitude_Birth', 'Longitude_Birth', 'Latitude_College', 'Longitude_College'])
# tqdm.pandas()
# for index, rows in tqdm_notebook(NFL.iterrows(), total = NFL.shape[0]):
    
#     try:
#         _ , (NFL['Latitude_Birth'], NFL['Longitude_Birth']) = NFL['Full Address'].apply(partial(geocode, timeout = 5))

#     except TypeError:
#         print(f'Birth Place: {rows["Place_Birth"] + ", " + rows["State"]}')
#         latitude_birth = np.nan
#         longitude_birth = np.nan
    
#     try:
#         _ , (NFL['Latitude_College'], NFL['Longitude_College'])  = NFL['College'].apply(partial(geocode, timeout = 5))
         
              
#     except TypeError:
#         print(f'College: {rows["College"] + " University"}')
#         latitude_college = np.nan 
#         longitude_college = np.nan
              
#     GPS.append([latitude_birth, longitude_birth, latitude_college, longitude_college])
    

# GPS.head()
    

In [89]:
tqdm_notebook.pandas()
# _ , (NFL['Latitude_Birth'], NFL['Longitude_Birth']) = NFL['Full Address'].progress_apply(partial(geocode, timeout = 5))
# _ , (NFL['Latitude_College'], NFL['Longitude_College'])  = NFL['College'].progress_apply(partial(geocode, timeout = 5))
         
    
NFL['Coords_Birth'] = NFL['Full Address'].progress_apply(partial(geocode,timeout = 5)).apply(lambda x: (x.latitude, x.longitude))
NFL['Coords_College'] = NFL['College'].progress_apply(partial(geocode,timeout = 5)).apply(lambda x: (x.latitude, x.longitude))


In [91]:
NFL[['Latitude_Birth', 'Longitude_Birth']] = pd.DataFrame(NFL['Coords_Birth'].tolist(), index=NFL.index)
NFL[['Latitude_College', 'Longitude_College']] = pd.DataFrame(NFL['Coords_College'].tolist(), index=NFL.index)
NFL.head()

,Name,Date_Birth,Place_Birth,State,College,Experience,HS,HOF,Full Address,Coords_Birth,Coords_College,Latitude_Birth,Longitude_Birth,Latitude_College,Longitude_College
0,Gene Ronzani,28/3/1909,Iron Mountain,MI,Marquette University,8 Seasons,NaN,NaN,"Iron Mountain, MI","(45.82074000000006, -88.06524999999993)","(43.038740000000075, -87.92803999999995)",45.82074,-88.06525,43.03874,-87.92804
1,Keith Molesworth,20/10/1905,Washington,IA,Monmouth University,7 Seasons,NaN,NaN,"Washington, IA","(41.29643000000004, -91.69114999999994)","(40.28006000000005, -74.00546999999995)",41.29643,-91.69115,40.28006,-74.00547
2,Carl Brumbaugh,22/9/1906,West Milton,OH,Florida University,8 Seasons,NaN,NaN,"West Milton, OH","(39.96402000000006, -84.32827999999995)","(30.26392000000004, -81.50550999999996)",39.96402,-84.32828,30.26392,-81.50551
3,Ken Strong,21/3/1906,West Haven,CT,N.Y.U. University,12 Seasons,NaN,1967.0,"West Haven, CT","(41.27228000000008, -72.94997999999998)","(43.03143000000006, -76.11844999999994)",41.27228,-72.94998,43.03143,-76.11845
4,George Corbett,14/6/1908,Dix,IL,Millikin University,7 Seasons,NaN,NaN,"Dix, IL","(38.43963000000008, -88.93774999999994)","(39.84111000000007, -88.97458999999998)",38.43963,-88.93775,39.84111,-88.97459


In [92]:
NFL.drop(columns=['Coords_Birth', 'Coords_College', 'Full Address'], inplace= True)
NFL.head()

,Name,Date_Birth,Place_Birth,State,College,Experience,HS,HOF,Latitude_Birth,Longitude_Birth,Latitude_College,Longitude_College
0,Gene Ronzani,28/3/1909,Iron Mountain,MI,Marquette University,8 Seasons,NaN,NaN,45.82074,-88.06525,43.03874,-87.92804
1,Keith Molesworth,20/10/1905,Washington,IA,Monmouth University,7 Seasons,NaN,NaN,41.29643,-91.69115,40.28006,-74.00547
2,Carl Brumbaugh,22/9/1906,West Milton,OH,Florida University,8 Seasons,NaN,NaN,39.96402,-84.32828,30.26392,-81.50551
3,Ken Strong,21/3/1906,West Haven,CT,N.Y.U. University,12 Seasons,NaN,1967.0,41.27228,-72.94998,43.03143,-76.11845
4,George Corbett,14/6/1908,Dix,IL,Millikin University,7 Seasons,NaN,NaN,38.43963,-88.93775,39.84111,-88.97459


In [93]:
NFL.to_csv("NFL-GPS.csv", index=False)